# Import bibliotek

In [4]:
import pandas as pd

# Import danych

In [5]:
data = pd.DataFrame({
    'gender': ['male', 'female', 'male', 'female', 'female'],
    'age': [20, 35, 50, 40, 25],
    'height': [170, 165, 180, 160, 175],
    'weight': [90, 65, 82, 70, 72],
    'seating_work': [1, 0, 1, 1, 0],
    'hypertension': [1, 0, 1, 0, 1],
})

data

,gender,age,height,weight,seating_work,hypertension
0,male,20,170,90,1,1
1,female,35,165,65,0,0
2,male,50,180,82,1,1
3,female,40,160,70,1,0
4,female,25,175,72,0,1


# Klasa Node

In [6]:
# Klasa Node reprezentuje węzeł w drzewie decyzyjnym
class Node:

    def __init__(
            self,
            feature=None,         # nazwa cechy, po której następuje podział (np. 'wiek', 'wzrost')
            split_value=None,     # wartość podziału (np. wiek > 30)
            is_discrete=False,    # czy cecha jest dyskretna (True/False) czy ciągła (np. wiek)
            left=None,            # lewe poddrzewo (spełniony warunek podziału)
            right=None,           # prawe poddrzewo (niespełniony warunek podziału)
            value=None,           # wartość przewidywana w liściu
            parent = None):       # rodzic węzła (None dla korzenia)
            # left, right i value jednocześnie nie mogą być różne od None
        self.feature = feature
        self.split_value = split_value
        self.is_discrete = is_discrete
        self.left = left
        self.right = right
        self.value = value
        self.parent = parent

    @property # wytłumaczyłam sobie niżej w tekście, czym jest property
    def is_leaf(self):
        # Sprawdza czy węzeł jest liściem (nie ma dzieci)
        return self.left is None and self.right is None
    
    @property
    def is_root(self):
        # Sprawdza czy węzeł jest korzeniem (nie ma rodzica)
        return self.parent is None
    
    def predict(self, x: pd.Series): # ok, x: to taki hint dla pythona, ale i tak nic sobie z nim nie zrobi, czyli to hint dla mnie, dziękuję
        # Metoda dokonująca predykcji dla pojedynczego przykładu x
        # x to wiersz danych jako pandas Series
        
        # Jeśli jesteśmy w liściu, zwracamy jego wartość
        if self.is_leaf:
            return self.value
            
        # Dla cech dyskretnych sprawdzamy równość
        if self.is_discrete:
            if self.split_value == x[self.feature]:
                return self.right.predict(x)
            else:
                return self.left.predict(x)

        # Dla cech ciągłych sprawdzamy czy wartość jest mniejsza/równa
        else:
            if x[self.feature] >= self.split_value:
                return self.right.predict(x)
            else:
                return self.left.predict(x)
            

In [7]:
def gini_impurity_for_feature(df: pd.DataFrame, feature: str, target: str="hypertension") -> float:
    if len(df[feature].unique()) == 1:
        return 9999
    if len(df[feature].unique()) == 2:
        # dokładnie ten sam kod co w ifie wyżej
        # tworzymy tablicę kontyngencji
        contingency = pd.crosstab(df[feature], df[target])
        # zliczamy liczebność - liczba wierszy w wejściowej df
        total = contingency.sum().sum()
        # obliczamy gini dla każdego wiersza czyli dla każdej kategorii
        gini_per_category = 1 - (contingency.div(contingency.sum(axis=1), axis=0) ** 2).sum(axis=1)
        # obliczamy gini dla całej kolumny jako  średnia ważona
        gini_total = (gini_per_category * (contingency.sum(axis=1) / total)).sum()
        if gini_total == 0:
            return 9999
        return gini_total 

In [8]:
def lowest_gini(df: pd.DataFrame, target: str, used_cols: list):
    # lista kolumn z df bez kolumny bez targetu
    features = [feature for feature in df.columns if feature != target and feature not in used_cols]
    # szukamy feature, dla którego gini impurity jest najmniejszy przy podziale wg targutu
    best_feature = min(features, key=lambda feature: gini_impurity_for_feature(df, feature, target)) # XDDDD wooooow
    return best_feature

In [11]:
def build_tree_(df, target, used_cols):
    # na razie zmienne tylko dla których len(df[feature].unique())==2
    # obsługujemy przypadek, że df od razu jest liściem:
# sprawdzić, czy df jest puste -> defensive coding
    if len(df[target].unique()) == 1:
        print("first if") # czyli mamy df z samymi 0 lub 1 w kolumnie "hypertension"
        return Node(value=df[target].iloc[0]) # bierzemy pierwszą wartość z Series, bo wszystkie takie same
    # sprawdzamy, czy zostały jeszcze jakieś kolumny niewykorzystane
    if len(used_cols) == len(df.columns) - 1:
        mode = df[target].mode()[0]
        return Node(value=mode)
    best_feature = lowest_gini(df, target, used_cols)
    used_cols.append(best_feature)
    split_value = sorted(df[best_feature].unique())[0] # jeden celowo żeby pasowało do moich rozkmin z zeszytu, bo "male" drugie na liście
    print("Split value:", split_value)
    is_discrete = True
    # dzielimy na dwie df w zależności od spełnienia warunku
    right_df = df[df[best_feature]==split_value]
    left_df = df[df[best_feature]!=split_value]
    # potwierdzimy sobie, co wyprintuje
    print(right_df)
    # Jeśli któraś gałąź pusta → liść z najczęstszą klasą
    if len(left_df) == 0 or len(right_df) == 0:
        majority = df[target].mode()[0]
        print("majority")
        return Node(value=majority)
    left_child = build_tree_(left_df, target, used_cols)
    right_child = build_tree_(right_df, target, used_cols)
    current_node = Node(feature=best_feature, split_value=split_value, is_discrete=is_discrete, left=left_child, right=right_child)
    left_child.parent = current_node
    right_child.parent = current_node
    return current_node

In [12]:
def build_tree(df: pd.DataFrame, target: str="hypertension"):
    used_cols = []
    return build_tree_(df, target, used_cols)

Próbujemy rozszerzyć działanie drzewka na zmienne, które przyjmują więcej niż dwie wartości i są mierzalne xd